In [3]:

!pip install snowflake-connector-python


In [4]:
!pip install "snowflake-connector-python[pandas]"

In [5]:
!pip install credentials

In [2]:
from snowflake.connector.pandas_tools import write_pandas
import snowflake.connector
import pandas as pd


In [7]:
import credentials

In [3]:
conn = snowflake.connector.connect(
    user = 'ashika',
    password = 'Cervello123#',
    account = 'bpwmwqd-bk67062',
    warehouse = 'compute_wh',
    database = 'RAW',
    schema = 'test',
)

In [4]:
cur = conn.cursor()

In [10]:
data = cur.execute('select * from users')

In [11]:
fdata = data.fetchall()

In [12]:
fdata

[(1, 'admin_user', 'admin@example.com', 'hashed_pw_1', 'admin'),
 (2, 'trainer_john', 'john_trainer@example.com', 'hashed_pw_2', 'trainer'),
 (3, 'student_anna', 'anna_student@example.com', 'hashed_pw_3', 'student'),
 (4, 'student_mark', 'mark_student@example.com', 'hashed_pw_4', 'student'),
 (5, 'trainer_emma', 'emma_trainer@example.com', 'hashed_pw_5', 'trainer')]

In [13]:
from pyspark.sql import SparkSession

spark = SparkSession.Builder().master('local[*]').appName("Pranav").getOrCreate()

ModuleNotFoundError: No module named 'pyspark'

In [28]:
columns = ['id','Name','email','role_id','role']

In [43]:
df = spark.createDataFrame(fdata, schema=columns)

In [44]:
df.show()

+---+------------+--------------------+-----------+-------+
| id|        Name|               email|    role_id|   role|
+---+------------+--------------------+-----------+-------+
|  1|  admin_user|   admin@example.com|hashed_pw_1|  admin|
|  2|trainer_john|john_trainer@exam...|hashed_pw_2|trainer|
|  3|student_anna|anna_student@exam...|hashed_pw_3|student|
|  4|student_mark|mark_student@exam...|hashed_pw_4|student|
|  5|trainer_emma|emma_trainer@exam...|hashed_pw_5|trainer|
+---+------------+--------------------+-----------+-------+



In [5]:
cur.execute("""
    SELECT table_name 
    FROM information_schema.tables
    WHERE table_schema = 'TEST' AND table_type = 'BASE TABLE'
""")

tables = cur.fetchall()
table_names = [table[0] for table in tables]

# Initialize an empty dictionary to store data from all tables
all_data = {}

# Fetch data from all tables
for table_name in table_names:
    cur.execute(f"SELECT * FROM {table_name} LIMIT 100")  # Limit the rows for simplicity
    data2 = cur.fetchall()
    df = pd.DataFrame(data2, columns=[col[0] for col in cur.description])
    all_data[table_name] = df


# Display the first few rows from all tables (for review)
for table, data2 in all_data.items():
    print(f"\nData from table: {table}")
    print(data2.head())


Data from table: CUSTOMERS
Empty DataFrame
Columns: [CUSTOMER_ID, FIRST_NAME, LAST_NAME, EMAIL, PHONE, ADDRESS, JOIN_DATE]
Index: []

Data from table: FEEDBACK
Empty DataFrame
Columns: [FEEDBACK_ID, USER_ID, PRODUCT_ID, RATING, COMMENTS, FEEDBACK_DATE]
Index: []

Data from table: INVENTORY
Empty DataFrame
Columns: [INVENTORY_ID, PRODUCT_ID, WAREHOUSE_LOCATION, QUANTITY_ON_HAND, LAST_RESTOCK_DATE]
Index: []

Data from table: ORDERS
Empty DataFrame
Columns: [ORDER_ID, USER_ID, ORDER_DATE, SHIPPING_ADDRESS, TOTAL_AMOUNT, ORDER_STATUS, PAYMENT_METHOD]
Index: []

Data from table: PAYMENTS
Empty DataFrame
Columns: [PAYMENT_ID, ORDER_ID, PAYMENT_DATE, PAYMENT_AMOUNT, PAYMENT_METHOD, PAYMENT_STATUS]
Index: []

Data from table: PRODUCTS
Empty DataFrame
Columns: [PRODUCT_ID, PRODUCT_NAME, CATEGORY, PRICE, STOCK_COUNT, RELEASE_DATE, IS_ACTIVE]
Index: []

Data from table: RETURNS
Empty DataFrame
Columns: [RETURN_ID, ORDER_ID, PRODUCT_ID, RETURN_DATE, REASON, REFUND_AMOUNT]
Index: []

Data from ta

In [6]:
cur.description

[ResultMetadata(name='SALE_ID', type_code=0, display_size=None, internal_size=None, precision=38, scale=0, is_nullable=False),
 ResultMetadata(name='PRODUCT_ID', type_code=0, display_size=None, internal_size=None, precision=38, scale=0, is_nullable=True),
 ResultMetadata(name='CUSTOMER_ID', type_code=0, display_size=None, internal_size=None, precision=38, scale=0, is_nullable=True),
 ResultMetadata(name='SALE_DATE', type_code=6, display_size=None, internal_size=None, precision=0, scale=9, is_nullable=True),
 ResultMetadata(name='QUANTITY_SOLD', type_code=0, display_size=None, internal_size=None, precision=38, scale=0, is_nullable=True),
 ResultMetadata(name='SALE_PRICE', type_code=0, display_size=None, internal_size=None, precision=10, scale=2, is_nullable=True),
 ResultMetadata(name='DISCOUNT', type_code=0, display_size=None, internal_size=None, precision=5, scale=2, is_nullable=True)]

In [7]:
all_data

{'CUSTOMERS': Empty DataFrame
 Columns: [CUSTOMER_ID, FIRST_NAME, LAST_NAME, EMAIL, PHONE, ADDRESS, JOIN_DATE]
 Index: [],
 'FEEDBACK': Empty DataFrame
 Columns: [FEEDBACK_ID, USER_ID, PRODUCT_ID, RATING, COMMENTS, FEEDBACK_DATE]
 Index: [],
 'INVENTORY': Empty DataFrame
 Columns: [INVENTORY_ID, PRODUCT_ID, WAREHOUSE_LOCATION, QUANTITY_ON_HAND, LAST_RESTOCK_DATE]
 Index: [],
 'ORDERS': Empty DataFrame
 Columns: [ORDER_ID, USER_ID, ORDER_DATE, SHIPPING_ADDRESS, TOTAL_AMOUNT, ORDER_STATUS, PAYMENT_METHOD]
 Index: [],
 'PAYMENTS': Empty DataFrame
 Columns: [PAYMENT_ID, ORDER_ID, PAYMENT_DATE, PAYMENT_AMOUNT, PAYMENT_METHOD, PAYMENT_STATUS]
 Index: [],
 'PRODUCTS': Empty DataFrame
 Columns: [PRODUCT_ID, PRODUCT_NAME, CATEGORY, PRICE, STOCK_COUNT, RELEASE_DATE, IS_ACTIVE]
 Index: [],
 'RETURNS': Empty DataFrame
 Columns: [RETURN_ID, ORDER_ID, PRODUCT_ID, RETURN_DATE, REASON, REFUND_AMOUNT]
 Index: [],
 'SHIPPING': Empty DataFrame
 Columns: [SHIPPING_ID, ORDER_ID, SHIPPING_METHOD, SHIPPING_D

In [ ]:
cur.execute("""
    select get_ddl('SCHEMA','TEST');
""")

ff = cur.fetchall()

ff

[('create or replace schema TEST;\n\ncreate or replace TABLE CUSTOMERS (\n\tCUSTOMER_ID NUMBER(38,0) NOT NULL,\n\tFIRST_NAME VARCHAR(16777216),\n\tLAST_NAME VARCHAR(16777216),\n\tEMAIL VARCHAR(16777216),\n\tPHONE VARCHAR(16777216),\n\tADDRESS VARCHAR(16777216),\n\tJOIN_DATE TIMESTAMP_LTZ(9),\n\tprimary key (CUSTOMER_ID)\n);\ncreate or replace TABLE FEEDBACK (\n\tFEEDBACK_ID NUMBER(38,0) NOT NULL,\n\tUSER_ID NUMBER(38,0),\n\tPRODUCT_ID NUMBER(38,0),\n\tRATING NUMBER(38,0),\n\tCOMMENTS VARCHAR(16777216),\n\tFEEDBACK_DATE TIMESTAMP_LTZ(9),\n\tprimary key (FEEDBACK_ID)\n);\ncreate or replace TABLE INVENTORY (\n\tINVENTORY_ID NUMBER(38,0) NOT NULL,\n\tPRODUCT_ID NUMBER(38,0),\n\tWAREHOUSE_LOCATION VARCHAR(16777216),\n\tQUANTITY_ON_HAND NUMBER(38,0),\n\tLAST_RESTOCK_DATE TIMESTAMP_LTZ(9),\n\tprimary key (INVENTORY_ID)\n);\ncreate or replace TABLE ORDERS (\n\tORDER_ID NUMBER(38,0) NOT NULL,\n\tUSER_ID NUMBER(38,0),\n\tORDER_DATE TIMESTAMP_LTZ(9),\n\tSHIPPING_ADDRESS VARCHAR(16777216),\n\tTOTA

In [6]:

cur.execute("""
    SELECT 
    TABLE_NAME, 
    COLUMN_NAME, 
    DATA_TYPE, 
    IS_NULLABLE, 
    COLUMN_DEFAULT 
FROM 
    INFORMATION_SCHEMA.COLUMNS
WHERE 
    TABLE_SCHEMA = 'TEST'
    order by table_name
;
""")

metadata = cur.fetchall()

metadata

[('CUSTOMERS', 'CUSTOMER_ID', 'NUMBER', 'NO', None),
 ('CUSTOMERS', 'FIRST_NAME', 'TEXT', 'YES', None),
 ('CUSTOMERS', 'LAST_NAME', 'TEXT', 'YES', None),
 ('CUSTOMERS', 'EMAIL', 'TEXT', 'YES', None),
 ('CUSTOMERS', 'PHONE', 'TEXT', 'YES', None),
 ('CUSTOMERS', 'ADDRESS', 'TEXT', 'YES', None),
 ('CUSTOMERS', 'JOIN_DATE', 'TIMESTAMP_LTZ', 'YES', None),
 ('FEEDBACK', 'FEEDBACK_DATE', 'TIMESTAMP_LTZ', 'YES', None),
 ('FEEDBACK', 'USER_ID', 'NUMBER', 'YES', None),
 ('FEEDBACK', 'PRODUCT_ID', 'NUMBER', 'YES', None),
 ('FEEDBACK', 'FEEDBACK_ID', 'NUMBER', 'NO', None),
 ('FEEDBACK', 'RATING', 'NUMBER', 'YES', None),
 ('FEEDBACK', 'COMMENTS', 'TEXT', 'YES', None),
 ('INVENTORY', 'LAST_RESTOCK_DATE', 'TIMESTAMP_LTZ', 'YES', None),
 ('INVENTORY', 'PRODUCT_ID', 'NUMBER', 'YES', None),
 ('INVENTORY', 'INVENTORY_ID', 'NUMBER', 'NO', None),
 ('INVENTORY', 'QUANTITY_ON_HAND', 'NUMBER', 'YES', None),
 ('INVENTORY', 'WAREHOUSE_LOCATION', 'TEXT', 'YES', None),
 ('ORDERS', 'PAYMENT_METHOD', 'TEXT', 'YES',

In [7]:
import json

In [150]:
tables

{'CUSTOMERS': [{'column_name': 'CUSTOMER_ID',
   'data_type': 'NUMBER',
   'is_nullable': 'NO'},
  {'column_name': 'FIRST_NAME', 'data_type': 'TEXT', 'is_nullable': 'YES'},
  {'column_name': 'LAST_NAME', 'data_type': 'TEXT', 'is_nullable': 'YES'},
  {'column_name': 'EMAIL', 'data_type': 'TEXT', 'is_nullable': 'YES'},
  {'column_name': 'PHONE', 'data_type': 'TEXT', 'is_nullable': 'YES'},
  {'column_name': 'ADDRESS', 'data_type': 'TEXT', 'is_nullable': 'YES'},
  {'column_name': 'JOIN_DATE',
   'data_type': 'TIMESTAMP_LTZ',
   'is_nullable': 'YES'}],
 'FEEDBACK': [{'column_name': 'FEEDBACK_DATE',
   'data_type': 'TIMESTAMP_LTZ',
   'is_nullable': 'YES'},
  {'column_name': 'USER_ID', 'data_type': 'NUMBER', 'is_nullable': 'YES'},
  {'column_name': 'PRODUCT_ID', 'data_type': 'NUMBER', 'is_nullable': 'YES'},
  {'column_name': 'FEEDBACK_ID', 'data_type': 'NUMBER', 'is_nullable': 'NO'},
  {'column_name': 'RATING', 'data_type': 'NUMBER', 'is_nullable': 'YES'},
  {'column_name': 'COMMENTS', 'data

In [149]:
tables = {}

# Loop through each column data tuple
for table_name, column_name, data_type, is_nullable, default_value in metadata:
    # Initialize a new table in the dictionary if it doesn't exist
    if table_name not in tables:
        tables[table_name] = []
    
    # Add a new entry for the column in the table
    tables[table_name].append({
        "column_name": column_name,
        "data_type": data_type,
        "is_nullable": is_nullable  # Using None as placeholder value
    })

# Convert the tables dictionary to JSON format
json_data = json.dumps(tables, indent=4)

# Output the JSON data
print(json_data)

{
    "CUSTOMERS": [
        {
            "column_name": "CUSTOMER_ID",
            "data_type": "NUMBER",
            "is_nullable": "NO"
        },
        {
            "column_name": "FIRST_NAME",
            "data_type": "TEXT",
            "is_nullable": "YES"
        },
        {
            "column_name": "LAST_NAME",
            "data_type": "TEXT",
            "is_nullable": "YES"
        },
        {
            "column_name": "EMAIL",
            "data_type": "TEXT",
            "is_nullable": "YES"
        },
        {
            "column_name": "PHONE",
            "data_type": "TEXT",
            "is_nullable": "YES"
        },
        {
            "column_name": "ADDRESS",
            "data_type": "TEXT",
            "is_nullable": "YES"
        },
        {
            "column_name": "JOIN_DATE",
            "data_type": "TIMESTAMP_LTZ",
            "is_nullable": "YES"
        }
    ],
    "FEEDBACK": [
        {
            "column_name": "FEEDBACK_DATE",
         

In [151]:
from langchain_openai import AzureChatOpenAI
import os

model = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_4o_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    openai_api_key = os.environ["AZURE_OPENAI_API_KEY"],
)

In [2]:
from langchain_core.prompts import PromptTemplate

template = (
    '''You are a data generator tasked with creating synthetic data. 
    Based on the following JSON metadata describing table structure and data types, generate sample data rows for each column. 
    Ensure the data adheres to the specified types, constraints, and formats. 
    Provide 10 rows of sample data in JSON array format, and ensure it is realistic and coherent.
    {json_data}
    '''
)

prompt = PromptTemplate.from_template(template)

In [152]:
from langchain_core.prompts import PromptTemplate

prompt_template = """
You are a data generator tasked with creating synthetic data. Based on the following JSON metadata describing table structure and data types, generate sample data rows for each column. 
- Adhere to the specified types, constraints, and formats.
- Provide 10 rows of sample data in JSON array format.
- Ensure the data is realistic and coherent.

Metadata:
{metadata}

Expected Output:
Provide 10 rows of JSON data for each table. use same format as metadata.
Provide the output in pure json format which I can parse as a json data to various platforms.
Generate json serializable data.

This is required format in which we require generated data.
please remove any additional content.
Provide only json data which is in curly braces.
Remove all line which consist ``` and json word.
"""

# Create the prompt
prompt = PromptTemplate(input_variables=["metadata"], template=prompt_template)
formatted_prompt = prompt.format(metadata=json_data)

# print(formatted_prompt)
response = model(formatted_prompt)

synthetic_data = response.content
# Display the result
print("Generated Synthetic Data:\n", synthetic_data)

# try:
#     data = json.loads(json_data)
# except json.JSONDecodeError:
#     print("Error: Failed to parse generated JSON data.")
#     data = []

# # Save to CSV if data is valid
# if data:
#     output_file = "synthetic_data.csv"
#     column_names = [col["name"] for col in json_data["columns"]]

#     with open(output_file, mode="w", newline="", encoding="utf-8") as file:
#         writer = csv.DictWriter(file, fieldnames=column_names)
#         writer.writeheader()
#         writer.writerows(data)

#     print(f"Data successfully saved to {output_file}")
# else:
#     print("No valid data generated.")

Generated Synthetic Data:
 {
    "CUSTOMERS": [
        {
            "CUSTOMER_ID": 1,
            "FIRST_NAME": "John",
            "LAST_NAME": "Doe",
            "EMAIL": "john.doe@example.com",
            "PHONE": "123-456-7890",
            "ADDRESS": "123 Elm St, Springfield, USA",
            "JOIN_DATE": "2023-01-15T08:30:00Z"
        },
        {
            "CUSTOMER_ID": 2,
            "FIRST_NAME": "Jane",
            "LAST_NAME": "Smith",
            "EMAIL": "jane.smith@example.com",
            "PHONE": "987-654-3210",
            "ADDRESS": "456 Oak St, Springfield, USA",
            "JOIN_DATE": "2023-02-20T09:00:00Z"
        },
        {
            "CUSTOMER_ID": 3,
            "FIRST_NAME": "Alice",
            "LAST_NAME": "Johnson",
            "EMAIL": "alice.j@example.com",
            "PHONE": "555-123-4567",
            "ADDRESS": "789 Pine St, Springfield, USA",
            "JOIN_DATE": "2023-03-10T10:15:00Z"
        },
        {
            "CUSTOMER_ID": 

In [153]:
import csv

In [154]:
f1 = synthetic_data.replace("```json", "").replace("```", "").strip()
f1

'{\n    "CUSTOMERS": [\n        {\n            "CUSTOMER_ID": 1,\n            "FIRST_NAME": "John",\n            "LAST_NAME": "Doe",\n            "EMAIL": "john.doe@example.com",\n            "PHONE": "123-456-7890",\n            "ADDRESS": "123 Elm St, Springfield, USA",\n            "JOIN_DATE": "2023-01-15T08:30:00Z"\n        },\n        {\n            "CUSTOMER_ID": 2,\n            "FIRST_NAME": "Jane",\n            "LAST_NAME": "Smith",\n            "EMAIL": "jane.smith@example.com",\n            "PHONE": "987-654-3210",\n            "ADDRESS": "456 Oak St, Springfield, USA",\n            "JOIN_DATE": "2023-02-20T09:00:00Z"\n        },\n        {\n            "CUSTOMER_ID": 3,\n            "FIRST_NAME": "Alice",\n            "LAST_NAME": "Johnson",\n            "EMAIL": "alice.j@example.com",\n            "PHONE": "555-123-4567",\n            "ADDRESS": "789 Pine St, Springfield, USA",\n            "JOIN_DATE": "2023-03-10T10:15:00Z"\n        },\n        {\n            "CUSTOMER_I

In [155]:
try:
    data1 = json.loads(f1)
except json.JSONDecodeError:
    print("Error: Failed to parse generated JSON data.")
    data1 = []

data1

{'CUSTOMERS': [{'CUSTOMER_ID': 1,
   'FIRST_NAME': 'John',
   'LAST_NAME': 'Doe',
   'EMAIL': 'john.doe@example.com',
   'PHONE': '123-456-7890',
   'ADDRESS': '123 Elm St, Springfield, USA',
   'JOIN_DATE': '2023-01-15T08:30:00Z'},
  {'CUSTOMER_ID': 2,
   'FIRST_NAME': 'Jane',
   'LAST_NAME': 'Smith',
   'EMAIL': 'jane.smith@example.com',
   'PHONE': '987-654-3210',
   'ADDRESS': '456 Oak St, Springfield, USA',
   'JOIN_DATE': '2023-02-20T09:00:00Z'},
  {'CUSTOMER_ID': 3,
   'FIRST_NAME': 'Alice',
   'LAST_NAME': 'Johnson',
   'EMAIL': 'alice.j@example.com',
   'PHONE': '555-123-4567',
   'ADDRESS': '789 Pine St, Springfield, USA',
   'JOIN_DATE': '2023-03-10T10:15:00Z'},
  {'CUSTOMER_ID': 4,
   'FIRST_NAME': 'Bob',
   'LAST_NAME': 'Brown',
   'EMAIL': 'bob.brown@example.com',
   'PHONE': None,
   'ADDRESS': '321 Maple St, Springfield, USA',
   'JOIN_DATE': None},
  {'CUSTOMER_ID': 5,
   'FIRST_NAME': 'Charlie',
   'LAST_NAME': 'Davis',
   'EMAIL': 'charlie.davis@example.com',
   'PHO

In [156]:
type(data1)

dict

In [157]:
import csv

# Data in JSON format
data = json.loads(f1)

output_dir = "csv_output2"
os.makedirs(output_dir, exist_ok=True)

# Process each table
for table_name, rows in data.items():
    if rows:  # Check if the table has data
        # Define output CSV file path
        output_csv_file = os.path.join(output_dir, f"{table_name}.csv")
        
        # Get column names from the first row
        column_names = rows[0].keys()

        # Write data to CSV file
        with open(output_csv_file, mode="w", newline="", encoding="utf-8") as file:
            writer = csv.DictWriter(file, fieldnames=column_names)
            
            # Write header and rows
            writer.writeheader()
            writer.writerows(rows)

        print(f"Table '{table_name}' saved to {output_csv_file}")
    else:
        print(f"Table '{table_name}' is empty. No file created.")

Table 'CUSTOMERS' saved to csv_output2\CUSTOMERS.csv
Table 'FEEDBACK' saved to csv_output2\FEEDBACK.csv
Table 'INVENTORY' saved to csv_output2\INVENTORY.csv
Table 'ORDERS' saved to csv_output2\ORDERS.csv
Table 'PAYMENTS' saved to csv_output2\PAYMENTS.csv
Table 'PRODUCTS' saved to csv_output2\PRODUCTS.csv
Table 'RETURNS' saved to csv_output2\RETURNS.csv
Table 'SALES' saved to csv_output2\SALES.csv
Table 'SHIPPING' saved to csv_output2\SHIPPING.csv
Table 'USERS' saved to csv_output2\USERS.csv
